In [ ]:
import json

with open("../data/eval_dataset.json", "rt") as f:
    eval_dataset = json.load(f)

In [ ]:
from transformers import pipeline

model_name: str = "../models/06_05_25"
# model_name: str = "meta-llama/Llama-3.2-1B-Instruct"

pl = pipeline("text-generation", model=model_name)
pl.tokenizer.pad_token = pl.tokenizer.eos_token # add a padding token, otherwise it raises an error


In [ ]:
pl("1+1 ?")

In [ ]:
eval_dataset[19]

In [ ]:
from wordllama import WordLlama

# Load pre-trained embeddings (truncate dimension to 64)
wl = WordLlama.load(trunc_dim=64)


def test_model(pl, dataset_elem):
    question = [dataset_elem["conversation"][0][0]]
    answer = pl(question)[0]['generated_text'][1]['content']
    sim = wl.similarity(dataset_elem["ground_truth"], answer)
    return {
        "acronym": dataset_elem["acronym"],
        "sim_with_ground_truth": abs(sim),
    }

import tqdm
test_result = []
for data_elem in tqdm.tqdm(eval_dataset):
    test_result.append(test_model(pl, data_elem))

In [ ]:
import pandas as pd
df = pd.DataFrame(test_result)

In [ ]:
pd.set_option("display.max_rows", 300)
df

In [ ]:
df.to_csv("./test_result_ft.csv")

In [ ]:
df_no_ft = pd.read_csv("test_result_no_ft.csv", index_col=0)
df_ft = pd.read_csv("test_result_ft.csv", index_col=0)

In [ ]:
dg = pd.DataFrame()
dg["acronym"] = df_ft["acronym"]
dg["sim_no_ft"] = df_no_ft["sim_with_ground_truth"]
dg["sim_ft"] = df_ft["sim_with_ground_truth"]

In [ ]:
dg

In [ ]:
dg.sim_ft.mean(), dg.sim_no_ft.mean()

In [ ]:
pl("What is ESF ?")